In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  

import pandas as pd
import numpy as np

from datetime import datetime, timedelta
import parseIntervalFiles as pif
import parseActivityFiles as paf
#pun intended :)
import consolidateFiles as cf
import datacleaning as cl

## ! Parse activity files and parse interval files are to be replaced with the corresponding database queries as soon as they are available 

## 0 - Pipeline configuration 
* set the input/output directories, user id and verbose level

In [2]:
verbose = True
    
DATA_PATH = "C:\\Users\\ju\\GDrive\\Projects\\HeRV\\Data\\" 
# DATA_PATH = "/home/ju/GDrive/Projects/HeRV/Data/"
    
RAW_PATH = DATA_PATH + "Raw"
PRE_PATH = DATA_PATH + "PreProcessed"     

## 1 - Extract sessions from raw data (intervals + annotations files)

### 1.1 - List activities with session start/stop datetime
Parses activities files and prints all errors found (activities without start or stop, invalid dates, etc)

In [3]:
sessions = []
for user in range(7):
    print ("------- parsing activity files for user ", user, " -------")
    user_sess = paf.get_user_sessions(user, dirname=RAW_PATH)
    print ("------- parsing interval files for user ", user, " -------")
    user_sess = cf.sessions_add_beats(sessions=user_sess, dirname=RAW_PATH+"\\"+str(user), verbose=False)
    sessions.extend(user_sess)

------- parsing activity files for user  0  -------
reading act170929.csv ... 
reading act170930.csv ... 
reading act171001.csv ... 
reading act171003.csv ... 
reading act171004.csv ... 
reading act171005.csv ... 
orphan start in: ['2017-10-05 11:08:11', 'start', 'focused-passive', 'sitting', '']
orphan start in: ['2017-10-05 12:54:00', 'start', 'eat', 'sitting', '']
orphan start in: ['2017-10-05 13:48:39', 'start', 'rest-active', 'sitting', '']
orphan start in: ['2017-10-05 13:57:37', 'start', 'movement', 'standing', '']
orphan start in: ['2017-10-05 17:55:44', 'start', 'focused-active', 'sitting', '']
orphan start in: ['2017-10-05 21:53:45', 'start', 'rest-active', 'sitting', '']
orphan start in: ['2017-10-05 23:47:45', 'start', 'focused-active', 'sitting', 'sitting on bed, installing and using pandas']
reading act171006.csv ... 
reading act171010.csv ... 
orphan start in: ['2017-10-10 23:23:37', 'start', 'leisure-passive', 'sitting']
reading act171011.csv ... 
orphan stop in: ['2017

In [37]:
df = pd.DataFrame(sessions)

### Include column with beats count for ease of use 

In [38]:
df['beatscount'] = df['rr'].apply(len)

In [39]:
df.sample(5)

,activity,duration,notes,posture,rr,start,stop,user,beatscount
248,focused-passive,840,NaN,sit,"[{'date': 2017-10-06 09:23:04, 'interval': 812...",2017-10-06 09:23:00,2017-10-06 09:37:00,2,1094
386,rest-passive,1080,taxi,sit,"[{'date': 2017-12-27 12:52:01, 'interval': 943...",2017-12-27 12:52:00,2017-12-27 13:10:00,5,1174
408,eat,4014,NaN,sit,"[{'date': 2018-01-01 21:09:53, 'interval': 642...",2018-01-01 21:09:52,2018-01-01 22:16:46,6,5483
223,sleep,3120,NaN,lie,"[{'date': 2017-09-30 02:07:29, 'interval': 801...",2017-09-30 02:07:00,2017-09-30 02:59:00,2,1735
224,sleep,900,NaN,lie,"[{'date': 2017-09-30 03:00:01, 'interval': 806...",2017-09-30 03:00:00,2017-09-30 03:15:00,2,925


## 2 - Cleaning data

### 2.1 Removing outliers and artifacts

(Note: For now, beats won't be saved to the output file, so, when loading the sessions file and getting
beats from DB again, for example for fragmentation, they will have to be removed again. But this is a reasonably low-cost op)

In [40]:
df['rr'] = df['rr'].apply(cl.clean_rr_series)

In [41]:
df['beatscount_clean'] = df['rr'].apply(len)

In [42]:
df['removed_artifacts'] = df['beatscount'] - df['beatscount_clean']
df['beatscount'] = df['beatscount_clean']
df = df.drop(['beatscount_clean'], axis=1)

### 2.1 Removing sessions without beats recorded or with too few of them

In [46]:
df = df[df['beatscount'] > 100]

In [47]:
df.describe()

,duration,user,beatscount,removed_artifacts
count,376.000000,376.000000,376.000000,376.000000
mean,2794.885638,1.500000,3471.965426,20.787234
std,5247.313682,1.596663,5966.710217,62.943720
min,88.000000,0.000000,105.000000,0.000000
25%,900.000000,0.000000,1084.750000,0.000000
50%,1299.000000,1.000000,1682.500000,1.000000
75%,2289.750000,2.000000,3201.500000,11.250000
max,34360.000000,6.000000,46856.000000,591.000000


### 2.3 Removing wrong activities and grouping the ones with too few examples

In [48]:
df.at[df['activity']=='exercise-low', 'activity'] = 'exercise'
df.at[df['activity']=='exercise-high', 'activity'] = 'exercise'
df.at[df['activity']=='focused-passive ', 'activity'] = 'focused-passive'
df = df[df['activity']!='not-recorded']

In [49]:
df.groupby('activity').count()['duration']

activity
eat                 43
exercise             6
focused-active      86
focused-passive     37
household-chores    26
leisure-active       6
leisure-passive     45
movement            38
rest-active         24
rest-passive        17
sleep               44
Name: duration, dtype: int64

## 3 - Aggregating data

In [73]:
dic = df.to_dict(orient='records')
for i in dic:
    i.update(cf.features_from_dic(i['rr']))
print(dic[0]['rmssd'])

239.316870462


In [76]:
df = pd.DataFrame(dic)
df.sample(3)

,activity,beatscount,duration,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,...,removed_artifacts,rmssd,rr,sdnn,sess_id,start,stop,total_power,user,vlf
302,focused-active,1713,1440,743.667299,30.097725,1727.174903,2.322510,69.902275,73.718710,824.269119,...,0,42.193909,"[{'date': 2017-10-30 14:08:00, 'interval': 821...",88.688059,331,2017-10-30 14:07:00,2017-10-30 14:31:00,3955.046988,2,1484.204786
277,movement,4940,3420,300.332988,27.769918,781.171772,2.601019,72.230082,85.841559,708.537449,...,1,29.661654,"[{'date': 2017-10-16 11:43:01, 'interval': 672...",83.203551,305,2017-10-16 11:43:00,2017-10-16 12:40:00,1760.950691,2,679.445931
128,focused-active,3425,2405,179.591500,28.749192,445.092140,2.478359,71.250808,83.828913,718.583066,...,2,25.591159,"[{'date': 2017-11-07 19:04:01, 'interval': 716...",44.310603,151,2017-11-07 19:04:00,2017-11-07 19:44:05,1152.212827,0,527.529187


In [77]:
df['sess_id'] = df.index
df_export = df.drop(['rr'], axis = 1)
df_export.sample(1)

,activity,beatscount,duration,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,...,posture,removed_artifacts,rmssd,sdnn,sess_id,start,stop,total_power,user,vlf
47,focused-passive,1947,1594,320.329992,33.848465,626.034921,1.954344,66.151535,91.512172,662.275295,...,sitting,1,24.692601,65.115,47,2017-10-11 17:34:00,2017-10-11 18:00:34,1594.858765,0,648.493852


In [79]:
df_export.describe()

,beatscount,duration,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,nn50,pnn50,removed_artifacts,rmssd,sdnn,sess_id,total_power,user,vlf
count,372.000000,372.000000,372.000000,372.000000,372.000000,372.000000,372.000000,372.000000,372.000000,372.000000,372.000000,372.000000,372.000000,372.000000,372.000000,372.000000,372.000000,372.000000
mean,3476.204301,2800.422043,1936.525319,40.697589,1715.507555,1.999893,59.302411,83.964623,751.873633,811.419355,17.805777,20.892473,63.726062,91.679895,185.500000,5081.110506,1.494624,1429.077633
std,5981.885123,5264.835437,3631.390059,16.649124,1646.257317,1.563038,16.649124,15.792345,130.671694,2447.199608,15.094030,63.242947,49.433136,42.510270,107.531391,5923.310106,1.604399,1228.188667
min,105.000000,88.000000,18.812437,8.241489,71.761212,0.172079,14.681505,54.190501,396.424172,0.000000,0.000000,0.000000,7.449832,21.083352,0.000000,322.718049,0.000000,109.131956
25%,1092.750000,900.000000,394.520353,28.530486,755.942013,0.866621,46.427243,74.127757,670.076237,96.250000,7.184725,0.000000,33.356262,64.314062,92.750000,1958.291435,0.000000,642.038785
50%,1695.000000,1309.500000,719.166158,37.651404,1258.428641,1.655945,62.348596,81.884192,743.376151,248.500000,14.018431,1.000000,47.948750,80.546499,185.500000,3282.537422,1.000000,1083.008861
75%,3201.500000,2289.750000,1489.195794,53.572757,1977.908942,2.505038,71.469514,90.847536,820.237852,492.250000,24.925427,11.250000,72.338857,107.471672,278.250000,5112.606398,2.000000,1738.757269
max,46856.000000,34360.000000,22206.761899,85.318495,12243.363436,11.133730,91.758511,155.541355,1144.581425,20895.000000,81.780538,591.000000,290.138931,253.656845,371.000000,36726.950587,6.000000,7350.357547


In [ ]:
df_export.to_excel(PRE_PATH + "\\sessions.xlsx")